In [2]:
import pandas as pd
import seaborn as sns
import os
import cv2
from tqdm import tqdm
import pydicom

In [3]:
def get_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16,16))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    bgr = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return bgr

In [4]:
covid_front_df = pd.read_pickle('./covid_df.pkl')
covid_front_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 0 to 126
Data columns (total 16 columns):
Patientid         76 non-null int64
offset            56 non-null float64
sex               67 non-null object
age               63 non-null float64
finding           76 non-null object
survival          29 non-null object
view              76 non-null object
date              76 non-null object
location          60 non-null object
filename          76 non-null object
doi               44 non-null object
 url              76 non-null object
license           14 non-null object
clinical notes    71 non-null object
other notes       32 non-null object
Unnamed: 15       0 non-null float64
dtypes: float64(3), int64(1), object(12)
memory usage: 10.1+ KB


In [5]:
rsna_normal = pd.read_pickle('./rsna_df.pkl')
rsna_normal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20672 entries, 0 to 30224
Data columns (total 8 columns):
patientId    20672 non-null object
x            0 non-null float64
y            0 non-null float64
width        0 non-null float64
height       0 non-null float64
Target       20672 non-null int64
filename     20672 non-null object
is_covid     20672 non-null int64
dtypes: float64(4), int64(2), object(2)
memory usage: 1.4+ MB


In [9]:
for filename in tqdm(covid_front_df.filename.unique()):
    img = cv2.imread('./covid-chestxray-dataset-master/images/'+filename)
    img = get_clahe(img)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:13<00:00,  5.79it/s]


In [10]:
for patientid in tqdm(rsna_normal.patientId.unique()):
    ds = pydicom.dcmread('./stage_2_train_images/'+patientid+'.dcm')
    pixel_array_numpy = ds.pixel_array
    cv2.imwrite('./images/'+patientid+'.jpeg', pixel_array_numpy)
    img = get_clahe(cv2.imread('./images/'+patientid+'.jpeg'))
    cv2.imwrite('./images/'+patientid+'.jpeg', img)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 20672/20672 [35:03<00:00,  9.51it/s]
